Code for LeNet used from:  
https://blog.paperspace.com/writing-lenet5-from-scratch-in-python/

In [11]:
# Load in relevant libraries, and alias where appropriate
import random
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Define relevant variables for the ML task
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 10

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
seed = 42
random.seed(seed)
torch.manual_seed(seed)

#Loading the dataset and preprocessing
train_dataset = torchvision.datasets.MNIST(root = './data',
                                           train = True,
                                           transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download = True)


test_dataset = torchvision.datasets.MNIST(root = './data',
                                          train = False,
                                          transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                          download=True)


train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

In [13]:
#Defining the convolutional neural network
class torch_model(nn.Module):
    def __init__(self, num_classes):
        super(torch_model, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [14]:
torch_model = torch_model(num_classes).to(device)

#Setting the loss function
cost = nn.CrossEntropyLoss()

#Setting the optimizer with the model parameters and learning rate
optimizer = torch.optim.Adam(torch_model.parameters(), lr=learning_rate)

#this is defined to print how many steps are remaining when training
total_step = len(train_loader)

In [15]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.to(device)
        labels = labels.to(device)
        
        #Forward pass
        outputs = torch_model(images)
        loss = cost(outputs, labels)
        	
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        		
        if (i+1) % 400 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
        		           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [400/938], Loss: 0.0407
Epoch [1/10], Step [800/938], Loss: 0.0856
Epoch [2/10], Step [400/938], Loss: 0.0437
Epoch [2/10], Step [800/938], Loss: 0.0484
Epoch [3/10], Step [400/938], Loss: 0.0792
Epoch [3/10], Step [800/938], Loss: 0.0173
Epoch [4/10], Step [400/938], Loss: 0.0021
Epoch [4/10], Step [800/938], Loss: 0.0082
Epoch [5/10], Step [400/938], Loss: 0.0175
Epoch [5/10], Step [800/938], Loss: 0.0216
Epoch [6/10], Step [400/938], Loss: 0.0305
Epoch [6/10], Step [800/938], Loss: 0.0042
Epoch [7/10], Step [400/938], Loss: 0.0141
Epoch [7/10], Step [800/938], Loss: 0.0124
Epoch [8/10], Step [400/938], Loss: 0.0063
Epoch [8/10], Step [800/938], Loss: 0.0010
Epoch [9/10], Step [400/938], Loss: 0.0016
Epoch [9/10], Step [800/938], Loss: 0.0600
Epoch [10/10], Step [400/938], Loss: 0.0024
Epoch [10/10], Step [800/938], Loss: 0.0053


In [16]:
# Assuming your model is already trained

# Specify the path where you want to save the model weights
model_save_path = "lenet5_model.pth"

# Save the model's state dictionary
torch.save(torch_model.state_dict(), model_save_path)
print(f"Model weights saved to {model_save_path}")

Model weights saved to lenet5_model.pth


In [17]:
# Specify the path where the model weights are saved
model_save_path = "lenet5_model.pth"

# Instantiate the model architecture
torch_model = torch_model(num_classes).to(device)

# Load the saved state dictionary into the model
torch_model.load_state_dict(torch.load(model_save_path))
torch_model.eval()  # Set the model to evaluation mode
print("Model weights loaded successfully")

# Now you can use the model for inference or further training


Model weights loaded successfully


In [18]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
  
with torch.no_grad(): # note this should not be left on for WB attacks
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = torch_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))
	 

Accuracy of the network on the 10000 test images: 99.07 %
